In [40]:
!apt-get install -y libreoffice-writer
!pip install gspread oauth2client pandas python-docx google-auth google-auth-oauthlib google-auth-httplib2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libreoffice-writer is already the newest version (1:7.3.7-0ubuntu0.22.04.5).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [41]:


import pandas as pd
from docx import Document
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import os
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import subprocess
from google.oauth2 import service_account
service_account_path = '/content/certificate-generation-check-4bb0a7114848.json'

credentials = service_account.Credentials.from_service_account_file(
    service_account_path,
    scopes=['https://www.googleapis.com/auth/drive.file']
)

drive_service = build('drive', 'v3', credentials=credentials)
file_path = '/content/Email - Sheet1 (1).csv'
df = pd.read_csv(file_path)

certificate_template_path = '/content/Certificte_template (1).docx'
def create_certificate(name, output_path):
    doc = Document(certificate_template_path)
    for paragraph in doc.paragraphs:
        if '[NAME]' in paragraph.text:
            paragraph.text = paragraph.text.replace('[NAME]', name)

    doc.save(output_path)

certificate_links = []

for index, row in df.iterrows():
    name = row['name']

    docx_path = f"{name}_certificate.docx"
    create_certificate(name, docx_path)

    pdf_path = f"{name}_certificate.pdf"
    subprocess.run(['soffice', '--headless', '--convert-to', 'pdf', docx_path, '--outdir', '/content'])


    file_metadata = {'name': pdf_path, 'parents': ['16ah9GxctJ84RQCqyH4_i2uXjwbESgH73']}
    media = MediaFileUpload(pdf_path, mimetype='application/pdf')
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    file_id = file.get('id')
    link = f"https://drive.google.com/file/d/{file_id}/view?usp=sharing"
    certificate_links.append(link)

    os.remove(docx_path)
    os.remove(pdf_path)

df['certificate_link'] = certificate_links
df.to_csv('updated_csv_file.csv', index=False)

#from google.colab import files
#.download('updated_csv_file.csv')


In [42]:
!pip install python-dotenv
import os
import smtplib
from email.message import EmailMessage
from email.utils import formataddr
from pathlib import Path

from dotenv import load_dotenv

PORT = 587
EMAIL_SERVER = "smtp.gmail.com"

sender_email = "YOUR_EMAIL"
password_email = "YOUR PASSWORD"

def send_email(subject, receiver_email, name, certificate_link):
    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = formataddr(("MeritOrchestrator Certificate", f"{sender_email}"))
    msg["To"] = receiver_email
    msg["BCC"] = sender_email

    msg.set_content(
        f"""\
        Hi {name},
        I hope you are well.
        Here is the certificate to your participation on Cloud Computing Seminar {certificate_link} .
        Best regards
        CCC, PEC.
        """
    )
    msg.add_alternative(
        f"""\
    <html>
      <body>
        <p>Hi {name},</p>
        <p>I hope you are well.</p>
        <p>Here is the certificate to your participation on Cloud Computing Seminar <strong> {certificate_link} </strong></p>
        <p>Best regards</p>
        <p>CHARAN TEJA</p>
      </body>
    </html>
    """,
        subtype="html",
    )

    with smtplib.SMTP(EMAIL_SERVER, PORT) as server:
        server.starttls()
        server.login(sender_email, password_email)
        server.sendmail(sender_email, receiver_email, msg.as_string())

if __name__ == "__main__":
    send_email(
        subject="Certificate of Merit",
        name="Charan Teja",
        receiver_email="palletiraju1983@gmail.com",
        certificate_link="drive.google.com"
    )
from datetime import date
import pandas as pd
def load_df(url):
    df = pd.read_csv(f"/content/updated_csv_file.csv")
    return df

def query_data_and_send_emails(df):
    present = date.today()
    email_counter = 0
    for _, row in df.iterrows():
        send_email(
                subject=f'Happy Participation {row["name"]}',
                receiver_email=row["email"],
                name=row["name"],
                certificate_link=row["certificate_link"]
            )
        email_counter += 1
    return f"Total Emails Sent: {email_counter}"




df = load_df(URL)
result = query_data_and_send_emails(df)
print(result)


Total Emails Sent: 2
